<h3>Qatar2022 Squads</h3>
It reads all squads from Qatar2022 WC then create a new clean dbase to use to enhanced data about each player.<br>
Initial thoughs: Web crawls to get place of birth for each player<br>


In [574]:
import pandas as pd    
import numpy as np
import scipy.stats as stats
import re
import datetime
from datetime import datetime, date, time, timedelta
from pandas.api.types import is_datetime64_any_dtype as is_datetime
import time

In [582]:
def read_html(base_site, parser='h'):
    #
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:107.0) Gecko/20100101 Firefox/107.0'}    
    #
    response = requests.get(base_site, headers=headers)
    #
    if not response.status_code == 200:
        return None
    try:
        #
        if (parser == 'l'):
            soup = BeautifulSoup(response.content,'lxml')
            
        if (parser == 'h'):
            soup = BeautifulSoup(response.content, "html.parser")
        return (soup)
    except:
        return None
    #
#./. Eo read_html   


def player_wiki1 (fname, lname, flen, sitein=0, debug=0):
    
    if (sitein == 0):
        player = fname + "_" + lname
        if (flen == 1): player = fname    
        if (debug==1): print (player)
        site= "https://en.wikipedia.org/wiki/" + player
    else:
        site = fname
    #
    pob = "dnk"
    #
    
    print(site)
    soup = read_html(site)
    if (soup is None):
        if (debug==1): print ("ERROR: Web site access failed") 
    elif (re.search("Place of birth", soup.prettify())):
        tables = soup.find_all('tbody')
        #
        for table in tables:
            if (re.search("Place of birth", str(table.contents))):
                trs = table.find_all('tr')
                found = 0
                for entry in trs:
                    #print(entry.get_text())
                    txt = entry.get_text()
                    if (re.search("Place of birth", txt)):
                        if (debug==1): print ("Found it", txt)
                        pob = txt
                        found = 1
                        break
                #./. eo for entry in trs
        #./. eo for looping thru tabels looking for PoB
        if (pob != "dnk"):
            pob = pob.replace("\n"," ")
            pob = pob.replace("Place of birth","")
            pob = re.sub(r"^\s+","", pob)
        #./. eo if pob was found
    return(pob)
#
#./. Eo player_wiki1
#

def qwant_scrap(fname, lname, nation, debug=0):
    site = "https://www.qwant.com/?q=" + fname + "+" + lname + "+" + nation + "+footballer+date+of+birth&t=web"
    print (site)
    #
    plink = None
    tsoup = None
    tsoup = read_html(site)
    #
    divs = tsoup.find_all("div", {"class": "Q8spj"})    
    for div in divs:
        try:
            if (re.search(r"https:\/\/\w+\.wikipedia\.org", str(div['domain']))):        
                a = div.find('a')
                plink = a['href']
                if (debug==1): print("Found:", a['href'])
                break
        except:
            return None
    return(plink)
#
#./. eo def qwant_scrap
#

### Read data

In [557]:
#
xls0  = pd.ExcelFile("SquadLists-English.xlsx")
df = pd.read_excel(xls0)
#
xls1  = pd.ExcelFile("32con.xlsx")
cty_df = pd.read_excel(xls1)


In [558]:
cty_df['Cty'].iloc[0]

'Argentina'

### Dataframe Cleaning

In [559]:

df.iloc[1]
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
df.reset_index(drop=True, inplace=True)


In [560]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 54 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1006 non-null   object 
 1   Unnamed: 1   831 non-null    object 
 2   Unnamed: 2   104 non-null    object 
 3   Unnamed: 3   727 non-null    object 
 4   Unnamed: 4   30 non-null     object 
 5   Unnamed: 5   264 non-null    object 
 6   Unnamed: 6   247 non-null    object 
 7   Unnamed: 7   207 non-null    object 
 8   Unnamed: 8   185 non-null    object 
 9   Unnamed: 9   25 non-null     object 
 10  Unnamed: 10  26 non-null     object 
 11  Unnamed: 11  51 non-null     object 
 12  Unnamed: 12  104 non-null    object 
 13  Unnamed: 13  156 non-null    object 
 14  Unnamed: 14  77 non-null     object 
 15  Unnamed: 15  104 non-null    object 
 16  Unnamed: 16  78 non-null     object 
 17  Unnamed: 17  104 non-null    object 
 18  Unnamed: 18  26 non-null     object 
 19  Unname

### Get data from files
* Read each row
* In each row, get data from each column
* Add data to a new dataframe.

I: In this particular file all data was in different columns, so I could not just drop columns w/ NANs

In [565]:
def rename_staff (entry):
    num = 0
    if (re.match("Head", entry)):        
        pos="HC"
        num=27
    if (re.match("Assistant", entry)):   pos="AC"     
    if (re.match("Goalkeeping",entry)): pos="GC"
    #
    return(pos, num)

cols = list(df.columns)
cty_idx = 0
col_cnt = 0
ply_cnt = 0
#
fifa_df = pd.DataFrame()
#
for t in list(df.index):  
    # 
    for c in cols:
        #
        row_txt = str(df[c].loc[t])
        if (re.match("Unnamed: 0", c)):
            col_cnt = 1
            if (re.match(r"1\b", row_txt)):
                cty = cty_df['Cty'].iloc[cty_idx]
                cty_idx += 1 
                num = 1
                typerow = 0
                print ("I: First Player", cty, num, cty_idx)
                
            elif (re.match(r"^[a-zA-Z]", row_txt) and not re.match(r"^DOB", row_txt)):
                #
                typerow = 1
                (pos, num) = rename_staff(row_txt)
                if num == 0: num = ply_cnt
                if num != 0: ply_cnt = num
                ply_cnt += 1
                
            elif (re.match(r"^\d", row_txt)): # Assume to be a num > 1
                typerow = 0
                num = df[c].loc[t]
        else:
            if (df[c].loc[t] == df[c].loc[t]):             
                if (col_cnt == 1):
                    if (typerow == 0): pos   = df[c].loc[t]
                    if (typerow == 1): pname = df[c].loc[t]                
                        
                if (col_cnt == 2): # Player name
                    if (typerow == 0): pname = df[c].loc[t]
                    if (typerow == 1): fname = df[c].loc[t]                
                        
                if (col_cnt == 3): # First name
                    if (typerow == 0): fname   = df[c].loc[t]
                    if (typerow == 1): lname = df[c].loc[t]                
                        
                if (col_cnt == 4): # Last name
                    if (typerow == 0): lname = df[c].loc[t]
                    if (typerow == 1): 
                        shirt = pos
                        height = 100
                        caps  = 0
                        goals = 0                        
                        dob = "10/10/1971"
                        club  = df[c].loc[t]     
                        # I: Append series to new df  
                        new_row = pd.Series(data={'Num':num, 'Country':cty, 'Position':pos, 'PName':pname, 'FName':fname,
                                              'LName':lname, 'Shirt':shirt, 'DoB':dob,'Club':club,'Height':height,
                                              'Caps':caps, 'Goals':goals}, name='S')    
                        fifa_df = fifa_df.append(new_row, ignore_index=False)
                        
                if (col_cnt == 5): # Shirt
                    shirt = df[c].loc[t]
                if (col_cnt == 6): # DoB
                    dob = df[c].loc[t]
                if (col_cnt == 7): # Club
                    club = df[c].loc[t]
                if (col_cnt == 8): # Height
                    height = df[c].loc[t]
                if (col_cnt == 9): # Caps
                    caps = df[c].loc[t]
                if (col_cnt == 10): # Goals 
                    goals = df[c].loc[t]
                    #
                    # I: Append series to new df  
                    new_row = pd.Series(data={'Num':num, 'Country':cty, 'Position':pos, 'PName':pname, 'FName':fname,
                                              'LName':lname, 'Shirt':shirt, 'DoB':dob,'Club':club,'Height':height,
                                              'Caps':caps, 'Goals':goals}, name='I')    
                    fifa_df = fifa_df.append(new_row, ignore_index=False)
                #
                col_cnt += 1
                
    


I: First Player Argentina 1 1
I: First Player Australia 1 2
I: First Player Belgium 1 3
I: First Player Brazil 1 4
I: First Player Cameroon 1 5
I: First Player Canada 1 6
I: First Player Costa Rica 1 7
I: First Player Croatia 1 8
I: First Player Denmark 1 9
I: First Player Ecuador 1 10
I: First Player England 1 11
I: First Player France 1 12
I: First Player Germany 1 13
I: First Player Ghana 1 14
I: First Player Iran 1 15
I: First Player Japan 1 16
I: First Player Korea Republic 1 17
I: First Player Mexico 1 18
I: First Player Morocco 1 19
I: First Player Netherlands 1 20
I: First Player Poland 1 21
I: First Player Portugal 1 22
I: First Player Qatar 1 23
I: First Player Saudi Arabia 1 24
I: First Player Senegal 1 25
I: First Player Serbia 1 26
I: First Player Spain 1 27
I: First Player Switzerland 1 28
I: First Player Tunisia 1 29
I: First Player Uruguay 1 30
I: First Player USA 1 31
I: First Player Wales 1 32


In [566]:
#
# Reorder the columns to match pdf. Country=Col0
fifa_df = fifa_df[['Country', 'Num', 'Position', 'PName', 'FName', 'LName', 'Shirt', 'DoB', 'Height', 'Club', 'Caps', 'Goals']]
#
# Drop the index I/S given by the Dframe creation
fifa_df.reset_index(drop=True, inplace=True)
#
fifa_df['DoB'] = fifa_df['DoB'].apply(lambda x:datetime.strptime(x,'%d/%m/%Y'))
#
#
def get_age (x):
    from datetime import date
    today = date.today()
    age = today.year - x.year - ((today.month, today.day) < (x.month, x.day))
    return(age)

fifa_df['Age'] = fifa_df['DoB'].apply(lambda x:get_age(x))
#
fifa_df['Num']    = fifa_df['Num'].astype(int)
fifa_df['Height'] = fifa_df['Height'].astype(int)
fifa_df['Caps']   = fifa_df['Caps'].astype(int)
fifa_df['Goals']  = fifa_df['Goals'].astype(int)
#
fifa_df["PoB"] = "dnk"
#
fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Country   983 non-null    object        
 1   Num       983 non-null    int64         
 2   Position  983 non-null    object        
 3   PName     983 non-null    object        
 4   FName     983 non-null    object        
 5   LName     983 non-null    object        
 6   Shirt     983 non-null    object        
 7   DoB       983 non-null    datetime64[ns]
 8   Height    983 non-null    int64         
 9   Club      983 non-null    object        
 10  Caps      983 non-null    int64         
 11  Goals     983 non-null    int64         
 12  Age       983 non-null    int64         
 13  PoB       983 non-null    object        
dtypes: datetime64[ns](1), int64(5), object(8)
memory usage: 107.6+ KB


In [583]:
#
# I: Add Place of Birth. It will attempt to find it in wiki first. If not, then use qwant to get
#    the wiki link. The rest will have to be done manually. Transfermarkit shows city but not country of birth
#    Result: From 983 players, this section found PoB for 98.7% of players ...only 12 to find manually
for t in list(fifa_df.index):  
    # 
    if (re.match(r"HC|AC|GC", fifa_df['Position'].loc[t])):
        pob = str(fifa_df['Club'].loc[t])
    else:
        nation = str(fifa_df['Country'].loc[t])
        nation = nation.lower()
        #
        pname = str(fifa_df['PName'].loc[t])
        pname = pname.lower()
        pname = pname.replace("  "," ")
        pname_arr = pname.split(" ")
        if (len(pname_arr) > 1):
            fname = pname_arr[1]
            lname = pname_arr[0]
        else: # Covers for single-name players like Brasil.
            fname = pname_arr[0]
            lname = ""
        #
        fname = fname.capitalize()
        lname = lname.capitalize()
        #
        print("\n", fname, ":", lname)
        #      
        pob = player_wiki1(fname, lname, len(pname_arr))                        
        #
        # I: If not match found thru wikipedia, try looking at qwant => wiki again
        if (pob == "dnk"):            
            if (prev_loop == 1): time.sleep(1) # 1 sec delay
            prev_loop = 1
            plink = qwant_scrap(fname, lname, nation)
            if (plink is not None):
                pob = player_wiki1(plink, lname, len(pname_arr), 1)             
        else:
            prev_loop = 0
    #./. eo if player or coach
    #
    # I: Add pob to fifa_df
    fifa_df['PoB'].loc[t] = pob
    print(pob)
#pob


 Franco : Armani
https://en.wikipedia.org/wiki/Franco_Armani
Found it Place of birth
Casilda, Santa Fe, Argentina
Casilda, Santa Fe, Argentina

 Juan : Foyth
https://en.wikipedia.org/wiki/Juan_Foyth
Found it Place of birth
La Plata, Argentina
La Plata, Argentina

 Nicolas : Tagliafico
https://en.wikipedia.org/wiki/Nicolas_Tagliafico
Found it Place of birth
Buenos Aires, Argentina
Buenos Aires, Argentina

 Gonzalo : Montiel
https://en.wikipedia.org/wiki/Gonzalo_Montiel
Found it Place of birth
González Catán, Argentina
González Catán, Argentina

 Leandro : Paredes
https://en.wikipedia.org/wiki/Leandro_Paredes
Found it Place of birth
San Justo, Argentina
San Justo, Argentina

 German : Pezzella
https://en.wikipedia.org/wiki/German_Pezzella
Found it Place of birth
Bahía Blanca, Argentina
Bahía Blanca, Argentina

 Paul : De
https://en.wikipedia.org/wiki/Paul_De
dnk
https://www.qwant.com/?q=Paul+De+argentina+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Rodrigo_De_Paul

Found it Place of birth
Bree, Belgium
Bree, Belgium

 Toby : Alderweireld
https://en.wikipedia.org/wiki/Toby_Alderweireld
Found it Place of birth
Antwerp, Belgium
Antwerp, Belgium

 Arthur : Theate
https://en.wikipedia.org/wiki/Arthur_Theate
Found it Place of birth
Liège, Belgium
Liège, Belgium

 Wout : Faes
https://en.wikipedia.org/wiki/Wout_Faes
Found it Place of birth
Mol, Belgium
Mol, Belgium

 Jan : Vertonghen
https://en.wikipedia.org/wiki/Jan_Vertonghen
Found it Place of birth
Sint-Niklaas, Belgium
Sint-Niklaas, Belgium

 Axel : Witsel
https://en.wikipedia.org/wiki/Axel_Witsel
Found it Place of birth
Liège, Belgium
Liège, Belgium

 Bruyne : De
https://en.wikipedia.org/wiki/Bruyne_De
dnk
https://www.qwant.com/?q=Bruyne+De+belgium+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Kevin_De_Bruyne
https://en.wikipedia.org/wiki/Kevin_De_Bruyne
Found it Place of birth
Drongen, Belgium[3]

 Youri : Tielemans
https://en.wikipedia.org/wiki/Youri_Tielemans
Found it Place 

Found: https://en.wikipedia.org/wiki/Vin%C3%ADcius_J%C3%BAnior
https://en.wikipedia.org/wiki/Vin%C3%ADcius_J%C3%BAnior
Found it Place of birth
São Gonçalo, Brazil[2]

 Rodrygo : 
https://en.wikipedia.org/wiki/Rodrygo
Found it Place of birth
Osasco, Brazil
Osasco, Brazil

 Ribeiro : Everton
https://en.wikipedia.org/wiki/Ribeiro_Everton
dnk
https://www.qwant.com/?q=Ribeiro+Everton+brazil+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/%C3%89verton_Ribeiro
https://en.wikipedia.org/wiki/%C3%89verton_Ribeiro
Found it Place of birth
Arujá, São Paulo, Brazil

 Ederson : 
https://en.wikipedia.org/wiki/Ederson
dnk
https://www.qwant.com/?q=Ederson++brazil+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Ederson_(footballer,_born_1993)
https://en.wikipedia.org/wiki/Ederson_(footballer,_born_1993)
Found it Place of birth
Osasco, Brazil

 Bremer : 
https://en.wikipedia.org/wiki/Bremer
dnk
https://www.qwant.com/?q=Bremer++brazil+footballer+date+of+birth&t=web
F

Found it Place of birth
Brampton, Ontario, Canada
Brampton, Ontario, Canada

 Ike : Ugbo
https://en.wikipedia.org/wiki/Ike_Ugbo
Found it Place of birth
Lewisham, England
Lewisham, England

 Atiba : Hutchinson
https://en.wikipedia.org/wiki/Atiba_Hutchinson
Found it Place of birth
Brampton, Ontario, Canada
Brampton, Ontario, Canada

 Mark-anthony : Kaye
https://en.wikipedia.org/wiki/Mark-anthony_Kaye
dnk
https://www.qwant.com/?q=Mark-anthony+Kaye+canada+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Mark-Anthony_Kaye
https://en.wikipedia.org/wiki/Mark-Anthony_Kaye
Found it Place of birth
Toronto, Ontario, Canada

 Ismael : Kone
https://en.wikipedia.org/wiki/Ismael_Kone
dnk
https://www.qwant.com/?q=Ismael+Kone+canada+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Isma%C3%ABl_Kon%C3%A9_(soccer,_born_2002)
https://en.wikipedia.org/wiki/Isma%C3%ABl_Kon%C3%A9_(soccer,_born_2002)
Found it Place of birth
Abidjan, Ivory Coast

 James : Pantemis
https://e

Found it Place of birth
Linz, Austria
Linz, Austria

 Andrej : Kramaric
https://en.wikipedia.org/wiki/Andrej_Kramaric
Found it Place of birth
Zagreb, SR Croatia, Yugoslavia
Zagreb, SR Croatia, Yugoslavia

 Luka : Modric
https://en.wikipedia.org/wiki/Luka_Modric
Found it Place of birth
Zadar, SR Croatia, SFR Yugoslavia
Zadar, SR Croatia, SFR Yugoslavia

 Marcelo : Brozovic
https://en.wikipedia.org/wiki/Marcelo_Brozovic
Found it Place of birth
Zagreb, Croatia
Zagreb, Croatia

 Ivo : Grbic
https://en.wikipedia.org/wiki/Ivo_Grbic
dnk
https://www.qwant.com/?q=Ivo+Grbic+croatia+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Ivo_Grbi%C4%87_(footballer)
https://en.wikipedia.org/wiki/Ivo_Grbi%C4%87_(footballer)
Found it Place of birth
Split, Croatia

 Nikola : Vlasic
https://en.wikipedia.org/wiki/Nikola_Vlasic
Found it Place of birth
Split, Croatia
Split, Croatia

 Marko : Livaja
https://en.wikipedia.org/wiki/Marko_Livaja
Found it Place of birth
Split, Croatia
Split, Croati

...

dnk
https://www.qwant.com/?q=Pathe+Ciss+senegal+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Path%C3%A9_Ciss
https://en.wikipedia.org/wiki/Path%C3%A9_Ciss
Found it Place of birth
Dakar, Senegal

 Fode : Ballo-toure
https://en.wikipedia.org/wiki/Fode_Ballo-toure
dnk
https://www.qwant.com/?q=Fode+Ballo-toure+senegal+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Fod%C3%A9_Ballo-Tour%C3%A9
https://en.wikipedia.org/wiki/Fod%C3%A9_Ballo-Tour%C3%A9
Found it Place of birth
Conflans-Sainte-Honorine, France

 Iliman : Ndiaye
https://en.wikipedia.org/wiki/Iliman_Ndiaye
Found it Place of birth
Rouen, France
Rouen, France

 Ismail : Jakobs
https://en.wikipedia.org/wiki/Ismail_Jakobs
Found it Place of birth
Cologne, Germany
Cologne, Germany

 Krepin : Diatta
https://en.wikipedia.org/wiki/Krepin_Diatta
Found it Place of birth
Dakar, Senegal[2]
Dakar, Senegal[2]

 Edouard : Mendy
https://en.wikipedia.org/wiki/Edouard_Mendy
Found it Place of birth
Montivilli

https://www.qwant.com/?q=Eric+Garcia+spain+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Eric_Garc%C3%ADa_(footballer,_born_2001)
https://en.wikipedia.org/wiki/Eric_Garc%C3%ADa_(footballer,_born_2001)
Found it Place of birth
Martorell, Spain

 Pau : Torres
https://en.wikipedia.org/wiki/Pau_Torres
Found it Place of birth
Villarreal, Spain
Villarreal, Spain

 Sergio : Busquets
https://en.wikipedia.org/wiki/Sergio_Busquets
Found it Place of birth
Sabadell, Spain
Sabadell, Spain

 Marcos : Llorente
https://en.wikipedia.org/wiki/Marcos_Llorente
Found it Place of birth
Madrid, Spain
Madrid, Spain

 Alvaro : Morata
https://en.wikipedia.org/wiki/Alvaro_Morata
Found it Place of birth
Madrid, Spain
Madrid, Spain

 Koke : 
https://en.wikipedia.org/wiki/Koke
dnk
https://www.qwant.com/?q=Koke++spain+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Koke_(footballer,_born_1992)
https://en.wikipedia.org/wiki/Koke_(footballer,_born_1992)
Found it Place of birth


Found it Place of birth
Cannes, France
Cannes, France

 Youssef : Msakni
https://en.wikipedia.org/wiki/Youssef_Msakni
Found it Place of birth
Tunis, Tunisia
Tunis, Tunisia

 Hannibal : Mejbri
https://en.wikipedia.org/wiki/Hannibal_Mejbri
Found it Place of birth
Ivry-sur-Seine, France
Ivry-sur-Seine, France

 Issam : Jebali
https://en.wikipedia.org/wiki/Issam_Jebali
Found it Place of birth
Majaz al Bab, Tunisia
Majaz al Bab, Tunisia

 Wahbi : Khazri
https://en.wikipedia.org/wiki/Wahbi_Khazri
Found it Place of birth
Ajaccio, Corsica, France
Ajaccio, Corsica, France

 Taha : Khenissi
https://en.wikipedia.org/wiki/Taha_Khenissi
dnk
https://www.qwant.com/?q=Taha+Khenissi+tunisia+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Taha_Yassine_Khenissi
https://en.wikipedia.org/wiki/Taha_Yassine_Khenissi
Found it Place of birth
Zarzis, Tunisia[1]

 Ali : Maaloul
https://en.wikipedia.org/wiki/Ali_Maaloul
Found it Place of birth
Sfax, Tunisia
Sfax, Tunisia

 Ferjani : Sassi
http

https://www.qwant.com/?q=Matt+Turner+usa+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Matt_Turner_(soccer)
https://en.wikipedia.org/wiki/Matt_Turner_(soccer)
Found it Place of birth
Park Ridge, New Jersey, United States

 Sergino : Dest
https://en.wikipedia.org/wiki/Sergino_Dest
Found it Place of birth
Almere, Netherlands[2]
Almere, Netherlands[2]

 Walker : Zimmerman
https://en.wikipedia.org/wiki/Walker_Zimmerman
Found it Place of birth
Lawrenceville, Georgia, U.S.
Lawrenceville, Georgia, U.S.

 Tyler : Adams
https://en.wikipedia.org/wiki/Tyler_Adams
Found it Place of birth
Wappinger, New York, U.S.
Wappinger, New York, U.S.

 Antonee : Robinson
https://en.wikipedia.org/wiki/Antonee_Robinson
Found it Place of birth
Milton Keynes, England
Milton Keynes, England

 Yunus : Musah
https://en.wikipedia.org/wiki/Yunus_Musah
Found it Place of birth
New York City, New York, United States
New York City, New York, United States

 Giovanni : Reyna
https://en.wikipedia.org/w

Found it Place of birth
Cardiff, Wales
Cardiff, Wales

 Rubin : Colwill
https://en.wikipedia.org/wiki/Rubin_Colwill
Found it Place of birth
Neath, Wales
Neath, Wales

 Matt : Smith
https://en.wikipedia.org/wiki/Matt_Smith
dnk
https://www.qwant.com/?q=Matt+Smith+wales+footballer+date+of+birth&t=web
Found: https://en.wikipedia.org/wiki/Matthew_Smith_(footballer,_born_1999)
https://en.wikipedia.org/wiki/Matthew_Smith_(footballer,_born_1999)
Found it Place of birth
Redditch, England


In [619]:
fifa_df['PoB'] = fifa_df['PoB'].replace("\[\d+\]+","", regex=True)

def get_country(x):
    nation = x
    x = re.sub(r",\s+", ",", x)
    x = re.sub(r"^\s+", "", x)
    if (re.search(",", x)):
        #print("Found ,")
        pob_arr = x.split(",")
        nation = pob_arr[-1]        
        #print(type(nation))
    return(nation)

fifa_df['PoB_nation'] = fifa_df['PoB'].apply(lambda x: get_country(x))

fifa_df.tail(20)

,Country,Num,Position,PName,FName,LName,Shirt,DoB,Height,Club,Caps,Goals,Age,PoB,PoB_nation
963,Wales,11,FW,BALE Gareth,Gareth Frank,BALE,BALE,1989-07-16,186,Los Angeles FC (USA),108,40,33,"Cardiff, Wales",Wales
964,Wales,12,GK,WARD Danny,Daniel,WARD,WARD,1993-06-22,191,Leicester City FC (ENG),26,0,29,dnk,dnk
965,Wales,13,FW,MOORE Kieffer,Kieffer Roberto Francisco,MOORE,MOORE,1992-08-08,196,Bournemouth AFC (ENG),28,9,30,"Torquay, England",England
966,Wales,14,DF,ROBERTS Connor,Connor Richard John,ROBERTS,ROBERTS,1995-09-23,175,Burnley FC (ENG),41,3,27,"Crynant, Wales",Wales
967,Wales,15,DF,AMPADU Ethan,Ethan Kwame Colm Raymond,AMPADU,AMPADU,2000-09-14,183,Spezia Calcio (ITA),37,0,22,"Exeter, England",England
968,Wales,16,MF,MORRELL Joe,Joseff John,MORRELL,MORRELL,1997-01-03,185,Portsmouth FC (ENG),30,0,25,"Ipswich, England",England
969,Wales,17,DF,LOCKYER Tom,Thomas Alun,LOCKYER,LOCKYER,1994-12-03,185,Luton Town FC (ENG),14,0,28,"Cardiff, Wales",Wales
970,Wales,18,MF,WILLIAMS Jonny,Jonathan Peter,WILLIAMS,J WILLIAMS,1993-10-09,168,Swindon Town FC (ENG),33,2,29,"Pembury, Kent, England",England
971,Wales,19,FW,HARRIS Mark,Thomas Mark,HARRIS,HARRIS,1998-12-29,182,Cardiff City FC (WAL),5,0,23,"Swansea, Wales",Wales
972,Wales,20,FW,JAMES Daniel,Daniel Owen,JAMES,JAMES,1997-11-10,171,Fulham FC (ENG),38,5,25,"Kingston upon Hull, England",England


In [620]:
fifa_df.to_excel("Players_Qatar2022.xlsx", sheet_name='Sheet', index=False) 